#### This is going to be an implementation of a Decision Tree 
as part of [deep-ML problem 20](https://www.deep-ml.com/problems/20).

The implementation is using the ID3 algorithm. 


In [165]:
import math
from collections import Counter

In [166]:
def calculate_entropy(labels: list) -> float:
    """Calculate the entropy of a list of labels."""
    P = [i/len(labels) for i in list(Counter(labels).values())]
    H =  sum(-p*math.log2(p) for p in P)
    return H

In [167]:
def calculate_information_gain(examples: list[dict], attr: str, target_attr: str) -> float:
    
    """
    Information gain measures the reduction in entropy achieved by splitting 
    on a particular attribute.
    Calculate the information gain of splitting on attr.
    """
    D_size = len(examples)
    labels = [example[target_attr] for example in examples]
    H = calculate_entropy(labels) # entropy of the whole set

    IG = H
    
    attr_tags_counts = Counter([example[attr] for example in examples])
    for value in attr_tags_counts.keys():
            subset = [example for example in examples if example[attr] == value]
            subset_labels = [example[target_attr] for example in subset]
            if subset_labels:
                IG -= (len(subset) / D_size) * calculate_entropy(subset_labels)
    return IG

In [168]:
def majority_class(examples: list[dict], target_attr: str) -> str:
    """Return the majority class. Break ties alphabetically."""
    labels = [example[target_attr] for example in examples]
    label_counts = Counter(labels)
    max_count = max(label_counts.values())
    
    # Get all classes with max count (tie handling)
    candidates = [label for label, count in label_counts.items() if count == max_count]
    
    # Return alphabetically first in case of tie
    return sorted(candidates)[0]

In [ ]:
def learn_decision_tree(examples: list[dict], attributes: list[str], target_attr: str) -> dict:
    """Build a decision tree using the ID3 algorithm."""
    # get the unique labels from the examples 
    unique_labels = list(Counter([example[target_attr] for example in examples]))
    if len(unique_labels) == 1: # leaf perfect split
        return  unique_labels[0]
    
    if len(attributes) == 0: # leaf no more possible split
        return majority_class(examples, target_attr)
    
    # use IG
    best_attr = max(attributes, key= lambda x: calculate_information_gain(examples, x, target_attr))

    tree = {best_attr:{}}

    remaining_attributes = [attr for attr in attributes if attr!=best_attr]

    best_attr_vals = Counter([example[best_attr] for example in examples]).keys()
    for val in best_attr_vals:
        subset = [example for example in examples if example[best_attr] == val]
        tree[best_attr][val] = learn_decision_tree(subset, remaining_attributes, target_attr)

    return tree
    

{'Outlook': {'Sunny': 'No',
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [158]:
best_attr_vals = Counter([example['Outlook'] for example in examples]).keys()

for val in best_attr_vals:
    print(len([example for example in examples if example['Outlook'] == val]))



2
2
3
